In [4]:
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from os import urandom

def create_account():
    print("Create an Instant Messaging Account:")
    username = input("Enter your username: ")
    password = input("Enter your password: ")

    # Derive a key from the password using PBKDF2
    salt = urandom(16)
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password.encode())

    return username, key, salt

def pad(message):
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(message.encode()) + padder.finalize()
    return padded_data

def unpad(message):
    unpadder = padding.PKCS7(128).unpadder()
    unpadded_data = unpadder.update(message) + unpadder.finalize()
    return unpadded_data.decode()

def encrypt_message(message, key, iv):
    # Create an AES cipher object
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # Encrypt the message
    padded_message = pad(message)
    ciphertext = encryptor.update(padded_message) + encryptor.finalize()

    return iv + ciphertext

def decrypt_message(encrypted_message, key):
    # Extract the IV from the encrypted message
    iv = encrypted_message[:16]
    ciphertext = encrypted_message[16:]

    # Create an AES cipher object
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()

    # Decrypt the message
    padded_message = decryptor.update(ciphertext) + decryptor.finalize()
    decrypted_message = unpad(padded_message)

    return decrypted_message

if __name__ == "__main__":
    # Create an account
    username, key, salt = create_account()
    print(f"Account created for user: {username}")

    # Encrypt a message
    message_to_encrypt = input("Enter the message to encrypt: ")
    iv = urandom(16)
    encrypted_message = encrypt_message(message_to_encrypt, key, iv)
    print(f"Encrypted message: {encrypted_message}")

    # Decrypt the message
    decrypted_message = decrypt_message(encrypted_message, key)
    print(f"Decrypted message: {decrypted_message}")


Create an Instant Messaging Account:
Enter your username: JSPM
Enter your password: jspm
Account created for user: JSPM
Enter the message to encrypt: RSCOE
Encrypted message: b'\x95~\xdb\xd5\x1a\xf9f9\x979\x01\x83\xbc\xa1q\x0c\xf99 \xd9R@/\xdb\x8c\x02s_\x98\x01\x00\xe4'
Decrypted message: RSCOE
